# Restricted Two Body Problem: Elliptical Orbits Around a Central Mass

This is the general case of a Keperian orbit.<br>
A light body (e.g. a planet) orbits a heavy central body (e.g. the sun).  The orbit is an ellipse with the primary at one focus.

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np
import matplotlib.pyplot as plt
import time

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
from r2b import VectorError
from orbital_element import make_data_orb_elt, make_dataset_elt_to_cfg
from orbital_element import OrbitalElementToConfig, make_model_elt_to_cfg

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [5]:
# Lightweight serialization
fname = '../pickle/orbital_element.pickle'
vartbl = load_vartbl(fname)

In [6]:
# Create small data set for orbital elements; dictionaries of numpy arrays
n = 100
a_min = 1.0
a_max = 2.0
e_max = 1.0
inc_max = np.pi/4.0
seed=42
elts, cart = make_data_orb_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, inc_max=inc_max, seed=seed)

In [7]:
# Create a tensorflow Dataset instance
batch_size = 64
ds_e2c = make_dataset_elt_to_cfg(n=n, a_min=a_min, a_max=a_max, e_max=e_max, inc_max=inc_max, seed=seed, batch_size=batch_size)

In [8]:
# Example batch
elts, cart = list(ds_e2c.take(1))[0]

a = elts['a']
e = elts['e']
inc = elts['inc']
Omega = elts['Omega']
omega = elts['omega']
f = elts['f']
mu = elts['mu']

q = cart['q']
v = cart['v']

print(f'Example batch sizes:')
print(f'a    = {a.shape}')
print(f'e    = {e.shape}')
print(f'inc  = {inc.shape}')
print(f'Omega= {Omega.shape}')
print(f'omega= {omega.shape}')
print(f'f    = {f.shape}')
print(f'q    = {q.shape}')
print(f'v    = {v.shape}')

Example batch sizes:
a    = (64,)
e    = (64,)
inc  = (64,)
Omega= (64,)
omega= (64,)
f    = (64,)
q    = (64, 3)
v    = (64, 3)


In [9]:
# Run the layer on the batch of orbital elements
inputs = (a, e, inc, Omega, omega, f, mu)
cart_rec = OrbitalElementToConfig()(inputs)

In [10]:
# Create a model mapping orbital elements to cartesian coordinates
model_e2c = make_model_elt_to_cfg()

In [11]:
# Inputs to compile this model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss')}

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0}

In [12]:
# Compile the model
model_e2c.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [13]:
# Verify that model matches rebound
model_e2c.evaluate(ds_e2c)

2/2 [==============================] - 0s 128ms/step - loss: 3.3443e-14 - q_loss: 2.3860e-14 - v_loss: 9.5832e-15


[3.3443230755786195e-14, 2.386005e-14, 9.58318e-15]

In [14]:
# Summary of the model mapping orbital elements to position
model_e2c.summary()

Model: "orbital_element_to_cartesian"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
a (InputLayer)                  [(None, 1)]          0                                            
__________________________________________________________________________________________________
e (InputLayer)                  [(None, 1)]          0                                            
__________________________________________________________________________________________________
inc (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
Omega (InputLayer)              [(None, 1)]          0                                            
_______________________________________________________________________